<a href="https://www.kaggle.com/code/enricofindley/spacex-rocket-landing-prediction?scriptVersionId=139669735" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Install & Importing Dependencies

In [ ]:
# Installing Dependencies
%pip install folium

# Importing Dependencies
import requests
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import folium
import pandas as pd
from folium.plugins import MarkerCluster,MousePosition
from folium.features import DivIcon

from sklearn import preprocessing
from sklearn.model_selection import train_test_split,GridSearchCV
# Logistic Regression classification algorithm
from sklearn.linear_model import LogisticRegression
# Support Vector Machine classification algorithm
from sklearn.svm import SVC
# Decision Tree classification algorithm
from sklearn.tree import DecisionTreeClassifier
# K Nearest Neighbors classification algorithm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

# Data Collecting

### There are 2 ways to collect the data:
* API
* Webscraping: https://kaggle.com/code/enricofindley/web-scraping-space-x-falcon-9-data

But in this notebook we will use API from SpaceX to collect data (see webscraping technique from link above)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data:
* From the <code>rocket</code> column we would like to learn the booster name.
* From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.
* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.
* From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [ ]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])
        
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])
        
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

Now let's start requesting rocket launch data from SpaceX API with the following URL:

In [ ]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

# Request API data and get Response
response = requests.get(spacex_url)

# Use json_normalize meethod to convert the json result into a dataframe
data = pd.json_normalize(response.json())

# Get the head of the dataframe
data.head()

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.

In [ ]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

Now we will get all the data using helper function we already created above.

In [ ]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

# Call getBoosterVersion
getBoosterVersion(data)
# Call getLaunchSite
getLaunchSite(data)
# Call getPayloadData
getPayloadData(data)
# Call getCoreData
getCoreData(data)

In [ ]:
# Create dictionary to accomodate the data parsed
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

# Create a data from launch_dict
launch_df = pd.DataFrame.from_dict(launch_dict)
# Show the head of the dataframe
launch_df.head()

### Next we have to do this task below to make this data to be useable:
* Filter the dataframe to only include `Falcon 9` launches
* Dealing with missing values

In [ ]:
# Filter the dataframe to only include Faclon 9
data_falcon9 = launch_df[launch_df['BoosterVersion']!='Falcon 1']
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))

# Replace missing value
# Calculate the mean value of PayloadMass column
mean = data_falcon9['PayloadMass'].mean()
# Replace the np.nan values with its mean value with replace() function
data_falcon9['PayloadMass'].replace(np.nan, mean, inplace=True)
data_falcon9

# Data Wrangling

#### This step is for preparing training label

In [ ]:
# Identify and calculate the percentage of the missing values in each attribute
print(data_falcon9.isnull().sum()/data_falcon9.count()*100)
print("\n")
# Identify which columns are numerical and categorical:
print(data_falcon9.dtypes)

In [ ]:
# Use the method .value_counts() on the column LaunchSite to determine the number of launches on each site: 
print(data_falcon9['LaunchSite'].value_counts(), "\n")

# Use the method .value_counts() to determine the number and occurrence of each orbit in the  column Orbit
print(data_falcon9['Orbit'].value_counts(),"\n")

# Use the method .value_counts() on the column Outcome to determine the number of landing_outcomes.
landing_outcomes = data_falcon9['Outcome'].value_counts()
print(landing_outcomes)

<code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully  landed to a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed to a drone ship. <code>None ASDS</code> and <code>None None</code> these represent a failure to land.

Now lets see the index of each landing outcome

In [ ]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

In [ ]:
# calculate landing_outcomes_percentage that success only
landing_outcomes_percentage = landing_outcomes/data_falcon9['Outcome'].count()*100
print("Success Percentage:",landing_outcomes_percentage[0]+landing_outcomes_percentage[2]+landing_outcomes_percentage[4])

We create a set of outcomes where the second stage did not land successfully and then make the label for training <code>Class</code>

In [ ]:
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
# landing_class = 0 if bad_outcome
# landing_class = 1 otherwise
data_falcon9['Class']=data_falcon9['Outcome'].apply(lambda x: 0 if x in bad_outcomes else 1)
data_falcon9.head()

# Save data for used later as label for machine learning models
data_label = data_falcon9

# Exploratory Data Analysis

### We are going to do task below to explore the data:
* Data visualization using matplotlib, seaborn, and folium
* Analysis each visualization

#### Folium Visualization

First we select relevant sub-colums first to feed the data to folium library.

In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = data_falcon9[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

Next we have to determine the start location when map is loading, in this case we choose nasa coordinate location.

In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    circle = folium.Circle([row['Latitude'], row['Longitude']], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['LaunchSite']))
    marker = folium.map.Marker(
        [row['Latitude'], row['Longitude']],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['LaunchSite'],
            )
        )
    site_map.add_child(marker)
    site_map.add_child(circle)
site_map

**Now, you can explore the map by zoom-in/out the marked areas, the marked location shows that the launch sites is nearing the coast.**

Next thing to do is we are going to mark the successful and failed launches for each sites.

In [ ]:
marker_cluster = MarkerCluster()
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
# assign marker color which red is failed launch and green is successful launches   
spacex_df['marker_color'] = spacex_df['Class'].apply(assign_marker_color)

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    marker = folium.Marker([record['Latitude'], record['Longitude']], icon=folium.Icon(color='white', icon_color=record['marker_color']))
    
    marker_cluster.add_child(marker)

site_map

Now try zoomed in and we can see from the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

Next we are going to calculate and draw distance between launches sites and its proximities.

In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)
site_map

Now u can see the coordinate of mouse when hovered on map showed on top right of the map.

Next try open notepad and write down the coordinate of launches sites and the nearest coastlines, in this case i use coordinates of KSC LC 39A launch sites.

But first we have to define function to help us calculate the distance between launch sites and the coastlines.

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
# first element is the coordinate of the launches sites and the second is coordinate of the coastlines
coordinates = [[28.60807,-80.60395],[28.61227,-80.59736]]
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
distance_coastline = calculate_distance(coordinates[0][0],coordinates[0][1],coordinates[1][0],coordinates[1][1])
print(distance_coastline)

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for examp
distance_marker = folium.Marker(
    coordinates[1],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 64; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)

As u zoomed in u can see the red colored text that show the distance between the launches sites and the nearest coastlines. But its still missing adding the lines so we can view it more easily.

In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=coordinates, weight=5)
site_map.add_child(lines)

Now we can see it more clearly, now since the launch sites supposed to be in close proximity to railways, highways and coastline, but not close to cities, next step is we are going to draw the lines between the launch sites and the nearest railways, highways and coastline.

A railway map symbol may look like this:
<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>

A highway map symbol may look like this:
<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>

A city map symbol may look like this:
<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>

In [ ]:
coordinate_dict = {
    'launch_site': coordinates[0],
    'city': [28.4011,-80.60489], # Cape Canaveral coordinates
    'highway': [28.60919,-80.59673],
    'railway': [28.61352, -80.59954]
}

# Calculating distance
distance_city = calculate_distance(coordinate_dict['launch_site'][0], coordinate_dict['launch_site'][1], coordinate_dict['city'][0], coordinate_dict['city'][1])
distance_highway = calculate_distance(coordinate_dict['launch_site'][0], coordinate_dict['launch_site'][1], coordinate_dict['highway'][0], coordinate_dict['highway'][1])
distance_railway = calculate_distance(coordinate_dict['launch_site'][0], coordinate_dict['launch_site'][1], coordinate_dict['railway'][0], coordinate_dict['railway'][1])

print(distance_city)
print(distance_highway)
print(distance_railway)

Draw marker

In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Display the distance on the marker
city_marker = folium.Marker(
    [coordinate_dict['city'][0], coordinate_dict['city'][1]],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
highway_marker = folium.Marker(
    [coordinate_dict['highway'][0], coordinate_dict['highway'][1]],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
        )
    )
railway_marker = folium.Marker(
    [coordinate_dict['railway'][0], coordinate_dict['railway'][1]],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
        )
    )
site_map.add_child(city_marker)
site_map.add_child(highway_marker)
site_map.add_child(railway_marker)

Draw lines

In [ ]:
city_lines=folium.PolyLine(locations=[[coordinate_dict['launch_site'][0], coordinate_dict['launch_site'][1]],[coordinate_dict['city'][0], coordinate_dict['city'][1]]], weight=5, color='red')
highway_lines=folium.PolyLine(locations=[[coordinate_dict['launch_site'][0], coordinate_dict['launch_site'][1]],[coordinate_dict['highway'][0], coordinate_dict['highway'][1]]], weight=5, color='green')
railway_lines=folium.PolyLine(locations=[[coordinate_dict['launch_site'][0], coordinate_dict['launch_site'][1]],[coordinate_dict['railway'][0], coordinate_dict['railway'][1]]], weight=5, color='gray')
site_map.add_child(city_lines)
site_map.add_child(highway_lines)
site_map.add_child(railway_lines)

Now we can see all distances that we need.

# Feature Engineering

**In this part we are going to prepare the feature for machine learning model we are going to implement:**
* First we are going to visually analyze first the relationship between feature except latitude and longitude which we are already analyzed.
* Then we do feature engineering.

First, let's try to see how the `FlightNumber` (indicating the continuous launch attempts.) and `Payload` variables would affect the launch outcome.

We can plot out the <code>FlightNumber</code> vs. <code>PayloadMass</code>and overlay the outcome of the launch. We see that as the flight number increases, the first stage is more likely to land successfully. The payload mass is also important; it seems the more massive the payload, the less likely the first stage will return.


In [ ]:
sns.catplot(y="PayloadMass", x="FlightNumber", hue="Class", data=data_falcon9, aspect = 5)
plt.xlabel("Flight Number",fontsize=20)
plt.ylabel("Pay load Mass (kg)",fontsize=20)
plt.show()

We see that different launch sites have different success rates.  <code>CCAFS LC-40</code>, has a success rate of 60 %, while  <code>KSC LC-39A</code> and <code>VAFB SLC 4E</code> has a success rate of 77%. Next, let's drill down to each site visualize its detailed launch records.

Use the function <code>catplot</code> to plot <code>FlightNumber</code> vs <code>LaunchSite</code>, set the  parameter <code>x</code>  parameter to <code>FlightNumber</code>,set the  <code>y</code> to <code>Launch Site</code> and set the parameter <code>hue</code> to <code>'class'</code>

In [ ]:
# Plot a scatter point chart with x axis to be Flight Number and y axis to be the launch site, and hue to be the class value
sns.catplot(x="FlightNumber", y="LaunchSite", hue="Class", data=data_falcon9, aspect = 5)

As the flight number is increasing, the success rate is increasing as well. The success rate is higher in CCAFS LC-40 and VAFB SLC 4E than KSC LC-39A.

We also want to observe if there is any relationship between launch sites and their payload mass.

In [ ]:
# Plot a scatter point chart with x axis to be Pay Load Mass (kg) and y axis to be the launch site, and hue to be the class value
sns.catplot(x="PayloadMass", y="LaunchSite", hue="Class", data=data_falcon9, aspect = 5)

Now if you observe Payload Vs. Launch Site scatter point chart you will find for the VAFB-SLC  launchsite there are no  rockets  launched for  heavypayload mass(greater than 10000).


Next, we want to visually check if there are any relationship between success rate and orbit type.


In [ ]:
# Visualize the relationship between success rate of each orbit type using a bar chart
(data_falcon9.groupby(['Orbit'])['Class'].mean()*100).plot(kind='bar', ylabel='Success Rate', xlabel='Orbit')

Orbit ES-L1, GEO, HEO, and SSO is the orbit with highest success rates. 

For each orbit, we want to see if there is any relationship between FlightNumber and Orbit type.


In [ ]:
# Plot a scatter point chart with x axis to be FlightNumber and y axis to be the Orbit, and hue to be the class value
sns.catplot(x="FlightNumber", y="Orbit", hue="Class", data=data_falcon9, aspect = 5)

You should see that in the LEO orbit the Success appears related to the number of flights; on the other hand, there seems to be no relationship between flight number when in GTO orbit.

Similarly, we can plot the Payload vs. Orbit scatter point charts to reveal the relationship between Payload and Orbit type


In [ ]:
# Plot a scatter point chart with x axis to be Payload and y axis to be the Orbit, and hue to be the class value
sns.catplot(x="PayloadMass", y="Orbit", hue="Class", data=data_falcon9, aspect = 5)

With heavy payloads the successful landing or positive landing rate are more for Polar,LEO and ISS.   

However for GTO we cannot distinguish this well as both positive landing rate and negative landing(unsuccessful mission) are both there here.

Visualize the launch success yearly trend with plot a line chart with x axis to be <code>Year</code> and y axis to be average success rate, to get the average launch success trend. 

In [ ]:
# A function to Extract years from the date 
year=[]
def Extract_year(date):
    for i in data_falcon9["Date"]:
        year.append(i.strftime("%Y"))
    return year
# Plot a line chart with x axis to be the extracted year and y axis to be the success rate
data_falcon9["year"]=Extract_year(data_falcon9["Date"])
(data_falcon9.groupby(['year'])['Class'].mean()*100).plot(kind='line', ylabel='Success Rate', xlabel='Year')

You can observe that the sucess rate since 2013 kept increasing till 2020


Finally we are going to engineered the feature before we can feed it to our machine learning models.

By now, you should obtain some preliminary insights about how each important variable would affect the success rate, we will select the features that will be used in success prediction in the future module.


In [ ]:
features = data_falcon9[['FlightNumber', 'PayloadMass', 'Orbit', 'LaunchSite', 'Flights', 'GridFins', 'Reused', 'Legs', 'LandingPad', 'Block', 'ReusedCount', 'Serial']]
features.head()

Use the function <code>get_dummies</code> and <code>features</code> dataframe to apply OneHotEncoder to the column <code>Orbits</code>, <code>LaunchSite</code>, <code>LandingPad</code>, and <code>Serial</code>. Assign the value to the variable <code>features_one_hot</code>, display the results using the method head.

In [ ]:
features_one_hot = pd.get_dummies(features, columns=['Orbit', 'LaunchSite', 'LandingPad', 'Serial'])
features_one_hot.head()

Now that our <code>features_one_hot</code> dataframe only contains numbers cast the entire dataframe to variable type <code>float64</code>

In [ ]:
features_one_hot = features_one_hot.astype('float64')

# Model Training and Testing

## In this part we are going to do these steps:
* Data Pre-processing such as transformation, normalization, and data splitting
* Training model
* Testing model

Define function to plot confusion matrix and load the feature and label

In [ ]:
def plot_confusion_matrix(y,y_predict):
    "this function plots the confusion matrix"

    cm = confusion_matrix(y, y_predict)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['did not land', 'land']); ax.yaxis.set_ticklabels(['did not land', 'landed']) 
    plt.show()

X = features_one_hot
Y = data_label['Class'].to_numpy()
X.head()

Normalize the feature and do splitting data into training and testing dataset

In [ ]:
# Normalize
transform = preprocessing.StandardScaler()
X = transform.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

**Train 4 machine learning model with hyperparameter tuning with GridSearch CV algorithm (logistic regression, SVM, decision tree, and KNN), after training plot the confusion matrix.**

**Logistic Regression**

In [ ]:
parameters ={'C':[0.01,0.1,1],
             'penalty':['l2'],
             'solver':['lbfgs']}
LR = LogisticRegression()
logreg_cv = GridSearchCV(LR, cv=10, param_grid=parameters)
logreg_cv.fit(X_train, Y_train)
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

print("Testing Accuracy", logreg_cv.score(X_test, Y_test))

yhat=logreg_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

**SVM (Support Vector Machine)**

In [ ]:
parameters = {'kernel':('linear', 'rbf','poly','rbf', 'sigmoid'),
              'C': np.logspace(-3, 3, 5),
              'gamma':np.logspace(-3, 3, 5)}
svm = SVC()

svm_cv = GridSearchCV(svm, cv=10, param_grid=parameters)
svm_cv.fit(X_train, Y_train)
print("tuned hpyerparameters :(best parameters) ",svm_cv.best_params_)
print("accuracy :",svm_cv.best_score_)

print("Testing Accuracy", svm_cv.score(X_test, Y_test))

yhat=svm_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

**Decision Tree**

In [ ]:
parameters = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}
tree = DecisionTreeClassifier()
tree_cv = GridSearchCV(tree, cv=10, param_grid=parameters)
tree_cv.fit(X_train, Y_train)
print("tuned hpyerparameters :(best parameters) ",tree_cv.best_params_)
print("accuracy :",tree_cv.best_score_)

print("Testing Accuracy", tree_cv.score(X_test, Y_test))

yhat = tree_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

KNN (K-Nearest Neighbors)

In [ ]:
parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}
KNN = KNeighborsClassifier()
knn_cv = GridSearchCV(KNN, cv=10, param_grid=parameters)
knn_cv.fit(X_train, Y_train)
print("tuned hpyerparameters :(best parameters) ",knn_cv.best_params_)
print("accuracy :",knn_cv.best_score_)

print("Testing Accuracy", knn_cv.score(X_test, Y_test))

yhat = knn_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

**Comparing each result**

In [ ]:
print("Model\t\tAccuracy\tTestAccuracy")
print("LogReg\t\t{}\t\t{}".format((logreg_cv.best_score_).round(5), logreg_cv.score(X_test, Y_test).round(5)))
print("SVM\t\t{}\t\t{}".format((svm_cv.best_score_).round(5), svm_cv.score(X_test, Y_test).round(5)))
print("Tree\t\t{}\t\t{}".format((tree_cv.best_score_).round(5), tree_cv.score(X_test, Y_test).round(5)))
print("KNN\t\t{}\t\t{}".format((knn_cv.best_score_).round(5), knn_cv.score(X_test, Y_test).round(5)))

comparison = {}

comparison['LogReg'] = {'Accuracy': logreg_cv.best_score_.round(5), 'TestAccuracy': logreg_cv.score(X_test, Y_test).round(5)}
comparison['SVM'] = {'Accuracy': svm_cv.best_score_.round(5), 'TestAccuracy': svm_cv.score(X_test, Y_test).round(5)}
comparison['Tree'] = {'Accuracy': tree_cv.best_score_.round(5), 'TestAccuracy': tree_cv.score(X_test, Y_test).round(5)}
comparison['KNN'] = {'Accuracy': knn_cv.best_score_.round(5), 'TestAccuracy': knn_cv.score(X_test, Y_test).round(5)}

# Conclusion
### It seems from 4 model we trained the model that best used for predicting if rocket landing success or not are the logistic regression model or support vector machine